In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName('Practice').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/15 11:44:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
df_spark=spark.read.csv("../data/Student_marks.csv",header=True,inferSchema=True)

In [10]:
df_spark.show()

+------+------+----+-------+-------+
|  Name|   Sex|Math|Science|English|
+------+------+----+-------+-------+
| Sunny|Female|  87|     91|     81|
| Bobby|  Male|  67|     56|     71|
|  Tina|Female|null|     54|     90|
| Reeta|  null|  71|   null|   null|
|Sachin|  Male|null|     67|     51|
| Honey|Female|  80|     76|   null|
+------+------+----+-------+-------+



In [34]:
df_spark.filter((df_spark['Math']>70)  & (df_spark['Math']<95)).select(['Math','Science']).show()

+----+-------+
|Math|Science|
+----+-------+
|  87|     91|
|  71|   null|
|  80|     76|
+----+-------+



In [35]:
df_spark.show()

+------+------+----+-------+-------+
|  Name|   Sex|Math|Science|English|
+------+------+----+-------+-------+
| Sunny|Female|  87|     91|     81|
| Bobby|  Male|  67|     56|     71|
|  Tina|Female|null|     54|     90|
| Reeta|  null|  71|   null|   null|
|Sachin|  Male|null|     67|     51|
| Honey|Female|  80|     76|   null|
+------+------+----+-------+-------+



In [36]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Math: integer (nullable = true)
 |-- Science: integer (nullable = true)
 |-- English: integer (nullable = true)



In [42]:
df_spark.describe().show()

+-------+-----+------+----------------+------------------+------------------+
|summary| Name|   Sex|            Math|           Science|           English|
+-------+-----+------+----------------+------------------+------------------+
|  count|    6|     5|               4|                 5|                 4|
|   mean| null|  null|           76.25|              68.8|             73.25|
| stddev| null|  null|8.99536917900909|15.254507530562892|16.740669042783207|
|    min|Bobby|Female|              67|                54|                51|
|    max| Tina|  Male|              87|                91|                90|
+-------+-----+------+----------------+------------------+------------------+



In [43]:
df_spark.withColumn('new_math',df_spark['Math']+5).show()

+------+------+----+-------+-------+--------+
|  Name|   Sex|Math|Science|English|new_math|
+------+------+----+-------+-------+--------+
| Sunny|Female|  87|     91|     81|      92|
| Bobby|  Male|  67|     56|     71|      72|
|  Tina|Female|null|     54|     90|    null|
| Reeta|  null|  71|   null|   null|      76|
|Sachin|  Male|null|     67|     51|    null|
| Honey|Female|  80|     76|   null|      85|
+------+------+----+-------+-------+--------+



In [52]:
df=spark.createDataFrame([(1,'India','Delhi',20000000), \
    (2,'Nepal','Kathmandu',None), \
        (3,'Srilanka','Colombo',None), \
            (4,'Bangladesh',None,10000000)],['id','Country','Capital','Population'])

In [53]:
df.show()

+---+----------+---------+----------+
| id|   Country|  Capital|Population|
+---+----------+---------+----------+
|  1|     India|    Delhi|  20000000|
|  2|     Nepal|Kathmandu|      null|
|  3|  Srilanka|  Colombo|      null|
|  4|Bangladesh|     null|  10000000|
+---+----------+---------+----------+



In [54]:
df.fillna({'Capital':'Dhaka','Population':3000}).show()

+---+----------+---------+----------+
| id|   Country|  Capital|Population|
+---+----------+---------+----------+
|  1|     India|    Delhi|  20000000|
|  2|     Nepal|Kathmandu|      3000|
|  3|  Srilanka|  Colombo|      3000|
|  4|Bangladesh|    Dhaka|  10000000|
+---+----------+---------+----------+



In [55]:
from pyspark.ml.feature import Imputer

In [64]:
imputer=Imputer(inputCol=('Population'),outputCol=('population_imputed')).setStrategy('median')

In [69]:
df_new=imputer.fit(df).transform(df)

In [70]:
df_new.show()

+---+----------+---------+----------+------------------+
| id|   Country|  Capital|Population|population_imputed|
+---+----------+---------+----------+------------------+
|  1|     India|    Delhi|  20000000|          20000000|
|  2|     Nepal|Kathmandu|      null|          10000000|
|  3|  Srilanka|  Colombo|      null|          10000000|
|  4|Bangladesh|     null|  10000000|          10000000|
+---+----------+---------+----------+------------------+



In [71]:
df_new.drop('population_imputed').show()

+---+----------+---------+----------+
| id|   Country|  Capital|Population|
+---+----------+---------+----------+
|  1|     India|    Delhi|  20000000|
|  2|     Nepal|Kathmandu|      null|
|  3|  Srilanka|  Colombo|      null|
|  4|Bangladesh|     null|  10000000|
+---+----------+---------+----------+



In [75]:
df_new.na.drop(how='all',thresh=3).show()

+---+----------+---------+----------+------------------+
| id|   Country|  Capital|Population|population_imputed|
+---+----------+---------+----------+------------------+
|  1|     India|    Delhi|  20000000|          20000000|
|  2|     Nepal|Kathmandu|      null|          10000000|
|  3|  Srilanka|  Colombo|      null|          10000000|
|  4|Bangladesh|     null|  10000000|          10000000|
+---+----------+---------+----------+------------------+



In [81]:
df_new.groupBy('Country').max().show()

+----------+-------+---------------+-----------------------+
|   Country|max(id)|max(Population)|max(population_imputed)|
+----------+-------+---------------+-----------------------+
|     India|      1|       20000000|               20000000|
|     Nepal|      2|           null|               10000000|
|  Srilanka|      3|           null|               10000000|
|Bangladesh|      4|       10000000|               10000000|
+----------+-------+---------------+-----------------------+

